In [7]:
import os
import json
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict

In [8]:
user_history_filepath = "/scratch/general/vast/u1471428/hugging_face_cache/user_history_data.json" 

In [9]:
json_file = open(user_history_filepath, 'r')
user_history = json.load(json_file)
json_file.close()

In [29]:
def preprocess_user_history(u_history):
    transactions = []
    for history in u_history.values():
        transaction = [entry['product_id'] for entry in history]
        transactions.append(transaction)
    return transactions

def apriori_pairs(transactions, min_support):
    min_support_count = min_support*len(transactions)
    print(min_support_count)
            
    #step2: Generate candidate pairs
    pair_counts = defaultdict(int)
    for transaction in transactions:
        for pair in combinations(transaction,2):
            if len(pair)<2:
                continue
            pair_counts[frozenset(pair)]+=1
            
    frequent_pairs = {pair: count for pair, count in pair_counts.items() if count >= min_support_count and len(pair)==2}
    return frequent_pairs

In [30]:
transactions = preprocess_user_history(user_history)
min_support = 0.0005
frequent_pairs = apriori_pairs(transactions, min_support)
print(len(frequent_pairs))
for pair, count in frequent_pairs.items():
    print(f"{set(pair)}: {count}")

22.0725
717
{'B0BQV3785S', 'B0C54J5D2B'}: 47
{'B09KLYL2ZX', 'B0C54J5D2B'}: 38
{'B0BQV3785S', 'B09KLYL2ZX'}: 95
{'B0BQV3785S', 'B0C68XP6K8'}: 32
{'B0BKV9ZGLH', 'B0C59SYPNT'}: 25
{'B0BQV3785S', 'B09YYZTK1W'}: 107
{'B09KLYL2ZX', 'B09YYZTK1W'}: 86
{'B09YYZTK1W', 'B0BMQ3G124'}: 50
{'B0BQV3785S', 'B0BMQ3G124'}: 88
{'B09KLYL2ZX', 'B0BMQ3G124'}: 67
{'B0BQV3785S', 'B07XM9DX9H'}: 63
{'B07XM9DX9H', 'B0BMQ3G124'}: 26
{'B09KLYL2ZX', 'B07XM9DX9H'}: 84
{'B09KLYL2ZX', 'B0BXX7LB93'}: 32
{'B0C3LV4FVT', 'B0BM5NFFF3'}: 23
{'B0BKV9ZGLH', 'B07XM9DX9H'}: 24
{'B0BKV9ZGLH', 'B0CDX5J8C6'}: 24
{'B09RTBP5T5', 'B07XM9DX9H'}: 38
{'B0CDX5J8C6', 'B07XM9DX9H'}: 44
{'B0052QYLUM', 'B07XM9DX9H'}: 23
{'B0C4W7L1S6', 'B0CDX5J8C6'}: 24
{'B09KVRXMHQ', 'B0BGYRXM3S'}: 29
{'B004JU0H6O', 'B0C1TK2V65'}: 41
{'B0C59SYPNT', 'B0C3RDQCL4'}: 44
{'B084SZ4BQB', 'B0C59SYPNT'}: 25
{'B0BY8YWJWG', 'B0C6BVZHT4'}: 24
{'B09KLYL2ZX', 'B0B63QBWFQ'}: 46
{'B0B63QBWFQ', 'B0C3WX29HS'}: 42
{'B09KLYL2ZX', 'B0C3WX29HS'}: 32
{'B0BQV3785S', 'B08K41XMBJ'}: 

In [33]:
json_compatible_data = {",".join(key): value for key, value in frequent_pairs.items()}

import json
with open('/scratch/general/vast/u1471428/hugging_face_cache/frequent_pairs_data.json', 'w') as json_file:
    json.dump(json_compatible_data, json_file, indent=4)
